In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json

# Load pre-trained main model
json_file = open("modelMain.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("modelMain.h5")

# Load pre-trained mns model
json_file1 = open("model-mns.json", "r")
model1_json = json_file1.read()
json_file1.close()
loaded_model_mns = model_from_json(model1_json)
loaded_model_mns.load_weights("model-mns.h5")

# Load pre-trained dru model
json_file2 = open("model-dru.json", "r")
model2_json = json_file2.read()
json_file2.close()
loaded_model_dru = model_from_json(model2_json)
loaded_model_dru.load_weights("model-dru.h5")

# Load pre-trained kv model
json_file3 = open("model-kv.json", "r")
model3_json = json_file3.read()
json_file3.close()
loaded_model_kv = model_from_json(model3_json)
loaded_model_kv.load_weights("model-kv.h5")

# Create a window to display the camera feed
cv2.namedWindow('Camera Feed', cv2.WINDOW_NORMAL)

# Define the ROI coordinates (x, y, width, height)
roi_coords = (700, 100, 400, 400)

# Define a function to preprocess the image
def preprocess_image(img):

    x, y, w, h = roi_coords
    img = img[y:y+h, x:x+w]

    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),2)
    th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    res = cv2.resize(res, (128, 128))
    
    return res

# Define a dictionary to map label indices to letters
label_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}

label_map2 = {0:'M', 1: 'N', 2: 'S'}

label_map3 = {0:'D', 1:'R', 2:'U'}

label_map4 = {0:'K', 1:'V'}

# Start capturing frames from the camera
cap = cv2.VideoCapture(0)
interrupt = -1
# Loop over the frames from the camera
while True:
    # Read the frame from the camera
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Get the ROI from the frame
    x, y, w, h = roi_coords
    roi = frame[y:y+h, x:x+w]

    # Preprocess the image
    img = preprocess_image(frame)

    # Feed the image to the model and get the predictions
    preds = loaded_model.predict(img.reshape(1,128,128,1))
    preds_mns = loaded_model_mns.predict(img.reshape(1,128,128,1))
    preds_dru = loaded_model_dru.predict(img.reshape(1,128,128,1))
    preds_kv = loaded_model_kv.predict(img.reshape(1,128,128,1))
    # Get the predicted class

    class_idx = np.argmax(preds)

    # Get the predicted letter
    letter = label_map[class_idx]


    #if letter is either of m,n or s, we pass it to another model 
    if(letter == 'M' or letter == 'N' or letter == 'S'):
        class_idx2 = np.argmax(preds_mns)
        letter = label_map2[class_idx2]
    
    if(letter == 'D' or letter == 'R' or letter == 'U'):
        class_idx3 = np.argmax(preds_dru)
        letter = label_map3[class_idx3]

    if(letter == 'K' or letter == 'V'):
        class_idx4 = np.argmax(preds_kv)
        letter = label_map4[class_idx4]

    # Display the predicted letter on the screen
    cv2.putText(frame, letter, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Draw a rectangle around the ROI on the frame
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the camera feed on the screen
    cv2.imshow('Camera Feed', frame)

    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break


# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()


/Users/ankitprasad/miniforge3/envs/tensorflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Init Plugin
Init Graph Optimizer
Init Kernel
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-08-05 10:02:19.822801: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-05 10:02:19.823086: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-08-05 10:02:23.477497: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-08-05 10:02:23.481836: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-08-05 10:02:25.919507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2023-08-05 10:02:26.872213: I tensorflow/core/grappler/

: 